In [39]:
# !pip install sqlite3
# !pip install time
# !pip install psutil
# !pip install os
# !pip install faker

In [1]:
import sqlite3
import time
import random
import psutil
import os
from faker import Faker
fake = Faker("pt_br")

In [2]:
# Função para monitorar memória e CPU
def monitorar_recursos():
    processo = psutil.Process(os.getpid())
    memoria = processo.memory_info().rss / (1024 * 1024)  # Memória em MB
    cpu = psutil.cpu_percent(interval=1)  # Uso de CPU em %
    return memoria, cpu


# Função para medir tempo de execução
def medir_tempo(func):
    def wrapper(*args, **kwargs):
        inicio = time.time()
        method, commit_type = func(*args, **kwargs)
        fim = time.time()
        tempo_total = fim - inicio  
        memoria, cpu = monitorar_recursos()
        # print(
        #     f"Tempo de execução: {tempo_total:.2f}s | Memória usada: {memoria:.2f}MB | CPU usada: {cpu}%"
        # )
        return method, commit_type, tempo_total, memoria, cpu

    return wrapper


def registrar_resultados(
    tempo_processamento, memoria, cpu, nr_registros, method, commit_type, conexao
):
    cursor_registro = conexao
    cursor_registro.execute(
        "insert into results (time, memori, cpu, nr_registros, method, commit_type) values (?,?,?,?,?,?)",
        (tempo_processamento, memoria, cpu, nr_registros, method, commit_type),
    )
    conexao.commit()


# Contar registros
def contar_registros(conexao):
    cursor = conexao.cursor()
    cursor.execute("SELECT COUNT(*) FROM teste;")
    resultado = cursor.fetchone()
    print(f"Total de registros: {resultado[0]}")

In [3]:
nome_banco = "sqlite_database.bd"
print(f"Banco de dados '{nome_banco}' criado.")
try:
    conexao = sqlite3.connect(nome_banco)
    cursor = conexao.cursor()
    cursor.executescript(
        """
        CREATE TABLE IF NOT EXISTS teste (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT,
            valor INTEGER
        );
        CREATE TABLE IF NOT EXISTS teste_batch (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT,
            valor INTEGER
        );
        CREATE TABLE IF NOT EXISTS results (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            time real,
            memori real,
            cpu real,
            nr_registros integer,
            method text,
            commit_type text CHECK(commit_type IN ('batch', 'per_row'))
        );
    """
    )
    conexao.commit()
    print("Tabelas criadas com sucesso.")
except Exception as e:
    print('Erro: ', e)
finally:
    conexao.close()

Banco de dados 'sqlite_database.bd' criado.
Tabelas criadas com sucesso.


In [4]:
@medir_tempo
def inserir_dados_execute(conexao, num_registros):
    cursor = conexao.cursor()
    cursor.execute("BEGIN TRANSACTION;")
    for _ in range(num_registros):
        nome = fake.name()
        valor = random.randint(1, 1000000)
        cursor.execute("INSERT INTO teste (nome, valor) VALUES (?, ?)", (nome, valor))
    conexao.commit()
    return 'execute', 'per_row'

In [5]:
@medir_tempo
def inserir_dados_execute_many(conexao, num_registros):
    cursor = conexao.cursor()
    cursor.execute("BEGIN TRANSACTION;")
    lst_insert_many = list()
    for _ in range(num_registros):
        nome = fake.name()
        valor = random.randint(1, 1000000)
        lst_insert_many.append((nome, valor))
        
    cursor.executemany("INSERT INTO teste_batch (nome, valor) VALUES (?, ?)", lst_insert_many)
    conexao.commit()
    return 'execute', 'batch'

In [49]:
# Testando o script
for nr_insert in range(10000):

    try:
        conexao = sqlite3.connect(nome_banco)
        method, commit_type, tempo_total, memoria, cpu = inserir_dados_execute(
            conexao, nr_insert
        ) 


        registrar_resultados(
            tempo_total, memoria, cpu, nr_insert, method, commit_type, conexao
        )
    except Exception as e:
        print(e)
    finally:
        conexao.close()

In [ ]:
# Testando o script
for nr_insert in range(10000):

    try:
        conexao = sqlite3.connect(nome_banco)
        method, commit_type, tempo_total, memoria, cpu = inserir_dados_execute_many(
            conexao, nr_insert
        ) 


        registrar_resultados(
            tempo_total, memoria, cpu, nr_insert, method, commit_type, conexao
        )
    except Exception as e:
        print(e)
    finally:
        conexao.close()

In [ ]:
monitorar_recursos()

(77.1484375, 6.6)

In [51]:
select_cursor = conexao.cursor()
select_cursor.execute(
    "SELECT COUNT(nome) as contagem_nome, sum(valor) as contagem_valor from teste where nome like 'F%'"
)
resultado = select_cursor.fetchall()
print(resultado)

ProgrammingError: Cannot operate on a closed database.

In [50]:
conexao.close()